<h1><strong>CSCI 4455&ndash; Fall 2022</strong></h1>

<h2><strong>Assignment 3 - Frequent Pattern Mining</strong></h2>

<h3><strong><span style="color:#cc3300;">Due: November 16, 08:59pm </span></strong></h3>

<h3><strong>Your name: Cody Kelly</strong></h3>

<ul>
<li style="text-align: justify;">Please note that you must do this assignment&nbsp;<span style="color: #cc3300;"><strong><u>individually</u></strong></span>. Using automatic tools, your code will be checked against other submissions and other existing resources (such as websites and books).</li>
<li style="text-align: justify;">This assignment is more extensive and might take longer than previous assignments to finish. <span style="color: #339966;"><strong><u>Please start early on.</u></strong></span></li>
<li style="text-align: justify;">Review the lecture notes before starting with this assignment. Then, thoroughly read this document before starting with the implementation or thinking about the solution.</li>
<li style="text-align: justify;">If you have technical questions about Python, please Google the error messages and share the error message alongside the solution that got it fixed on Microsoft Teams, as your classmates may run into the same issues.</li>
<li style="text-align: justify;">Check Canvas regularly for possible clarifications and updates.</li>
<li style="text-align: justify;">There are libraries and scripts for frequent pattern mining, but you are <span style="color: #cc3300;"><strong>prohibited to use these existing resources</strong></span>, which means you <span style="color: #cc3300;"><strong>cannot include public libraries</strong></span>, or <span style="color: #cc3300;"><strong>modify existing programs. Do not import any libraries. All the needed one are already imported for you.</strong></span> since the purpose   of   this   programming   assignment   is   to   help you understand and implement frequent pattern mining algorithms. You need to develop your code from scratch.</li>
</ul>


<h2>Dataset</h2>

<p style="text-align: justify;">A dataset is provided with this assignment that contains retail market basket data from an anonymous Belgian retail store. 
    
The retail dataset was originally used in the following paper: <a href="http://fimi.uantwerpen.be/data/retail.pdf">paper link.</a>

    
Each line in the .txt file shows the items purchased in one transaction (transaction IDs are not provided), and items are separated by a space.    
</p>

<h2>Implementation</h2>

<p style="text-align: justify;">Please consider the following in your implementation:<br>
    
<ul>

<li style="text-align: justify;">You are not allowed to use frequent itemset mining libraries and need to implement your code from scratch. However, you are allowed to use Python built-in functions, such as min, max, average, map, apply, reduce, etc.</li>
<li style="text-align: justify;">Ensure that the cells in your Notebook are ordered correctly so that the “run all” option can run all cells without running to dependency issues.</li>
</ul>

<span style="text-align: justify;"><strong><u>Hint</u></strong> Running the code with the provided dataset may take a long time. Moreover, verifying the correctness of your code with the provided dataset is not easy since it contains many transactions. Therefore, you should use a small dataset with a few transactions during the implementation to test your code fast and verify the results. However, note that you may obtain correct results for a small dataset, even though your code may have bugs, so eventually, you should try your code with the larger dataset.
</span>
</p>

In [14]:
import matplotlib.pyplot as plt
import collections
import math
from itertools import combinations
import time
import csv
from anytree import Node, RenderTree


dataset_name =   'retail.txt'     # dataset_name: that is the dataset name (such as dataset.txt)
patterns_path =  'patterns.csv'   # the file name that will store the frequent patterns
negative_path = 'negative.csv'    # the file name that will store the negatively correlated items
rules_path =     'rules.csv'      # the file name that will store the maximal patterns
maximal_path =   'maximal.csv'    # the file name that will store the association rules
min_support =     0.0015          # minimum support threshold which is in percentage format (for example 0.2 means 20%)
min_confidence =  0.9             # the minimum confidence threshold in the percentage format

<p>Note that in the rest of this Notebook, support count refers to the raw frequency (i.e., the number of times an itemsets occurs in a database), while support is the normalized support count which is equal to $\frac{\text{support count}}{\text{the number of transactions}}$ </p>

<h2>Implement the FP-Growth Algorithm (30 Points)</h2>

<p style="text-align: justify;">Implement the FP-Growth algorithm, as discussed in the lecture notes. Display the total execution time, and the total number of frequent patterns in the following format (the provided values are hypothetical):

    Execution time: 20 seconds
    Frequent itemsets: 10,123
    
Each line must contain one pattern, its support count, and support. For example, the pattern A-B-C with support count 4 and support 0.5 must be saved in one line as A-B-C,4,0.5 in the <i><u>patterns.csv</u></i> file.
</p>

In [15]:
support_counts = {}
transactions = []
total_items = 0

with open(dataset_name) as file:
    for line in file:
        transaction = []
        for item in line.split():
            intitem = int(item)
            if intitem not in support_counts:
                support_counts[intitem] = 1
            else:
                support_counts[intitem] += 1
            transaction.append(intitem)
            total_items += 1
        transactions.append(transaction)

# sorted_support_counts = [k for k, v in sorted(support_counts.items(), key=lambda item: item[1], reverse=True) if v > round(min_support * len(items))]

for i in range(len(transactions)):
    transaction = transactions[i]
    transactions[i] = ([item for item in sorted(transaction, key=lambda x:support_counts[x], reverse=True) if support_counts[item] > round(min_support * total_items)])
    
print(transactions)



[[3], [32], [], [39, 38, 41, 36], [39, 48, 38, 47], [39, 48, 38, 56], [41, 32], [39, 48, 3], [65, 66], [32], [48], [39, 78, 79], [39, 48, 38, 41, 36, 79], [], [41], [39, 48, 89], [39, 48, 38, 36, 89], [39, 41], [39, 38, 41], [39], [], [48], [39, 48, 140], [39], [39, 38, 56], [48], [39, 48, 41], [], [39, 48, 38], [39, 48, 41, 32], [39, 48, 38, 32, 47, 179], [39, 186], [48, 38, 41, 36, 140], [39, 48, 186, 193], [39], [39, 65, 193], [179], [225], [39, 48, 41, 230], [39, 38, 36, 242], [39], [39, 48, 41], [39, 48, 65], [48, 230], [39, 48, 242, 66, 78], [39, 48], [39, 38, 36, 225], [39, 242]]


In [16]:
class Node:
    def __init__(self, tid, parent):
        self.tid = tid
        self.count = 0
        self.parent = parent
        self.children = []
    def __str__(self):
        print(f"tid: {self.tid}, children: {[child.tid for child in self.children]}")
        for child in self.children:
            print(child)

root_node = Node(-1, None)
for transaction in transactions:
    current_node = root_node
    for item in transaction:
        child_found = False
        for child in current_node.children:
            if child.tid == item:
                child.count += 1
                current_node = child
                child_found = True
                break
        if not child_found:
            new_child = Node(item, current_node)
            new_child.count += 1
            current_node.children.append(new_child)

print(root_node)
        

tid: 193, children: []
tid: 39, children: [48, 38, 47, 78, 79, 41, 186, 65, 193, 242]
tid: 48, children: [38, 56, 3, 89, 140, 41, 186, 193, 65, 242, 66, 78]
tid: 38, children: [41, 36, 79, 32, 47, 179]
tid: 41, children: []
tid: 242, children: []
tid: -1, children: [3, 32, 39, 38, 41, 36, 65, 66, 48, 179, 225]
tid: 3, children: []
tid: -1, children: [3, 32, 39, 38, 41, 36, 65, 66, 48, 179, 225]
tid: 3, children: []


TypeError: __str__ returned non-string (type NoneType)

<h2>Find the negative correlated itemsets (10 Points)</h2>

<p style="text-align: justify;">Find the items (with length 1) that are negatively correlated among all the generated frequent items. Two frequent items, X and Y, are negatively correlated if:
support{X,Y} $\lt$ support{X} * support{Y}. Hint: Find the frequent items (length 1) in your patterns.csv file; then for each item you have, using the negatively correlated items' formula find the negatively correlated items for it. </p>

<p style="text-align: justify;">
Each line in your <i>negative.csv</i> output file must contain two items.
</p>


In [ ]:
# write the code to find the negatively correlated items here
# you can insert more cells below, if needed

<h2>Find Maximal Itemsets (20 Points)</h2>

<p style="text-align: justify;">Find the maximal itemset among all the generated frequent patterns. You do not need to implement a maximal itemset mining algorithm (such as the Charm algorithm that allows for mining maximal itemsets without mining the frequent itemsets). Instead, you can simply iterate through the frequent itemsets and identify the maximal ones.

<p style="text-align: justify;">Finally, display the number of maximal patterns and the compression ratio in the following format (the provided values are hypothetical):

    Frequent itemsets: 10,000
    Maximal patterns: 1,500
    Compression ratio: 85%

As before, each line in your <i>maximal.csv</i> output file must contain one pattern, its support count, and support.
</p>


<p style="text-align: justify;"><strong>Hint</strong>: Iterate through the frequent itemsets with the reverse order that were mined in the previous step, i.e., from the longest itemset(s) to shorter one(s). </p>


In [ ]:
# wrtie the code to implement and find maximal itemsets here
# you can insert more cells below, if needed

<h2>Mine the Association Rules (25 Points)</h2>


<p style="text-align: justify;">Implement association rule mining to generate strong association rules from the frequent itemsets generated in Step 1 using the below algorithm. Display the confidence for the top 20 rules with the highest confidence. Moreover, report the execution time as well as the number of strong rules in the following format (the provided values are hypothetical): </p>
    
    Execution time: 12 secodns 
    Strong rules: 2,300
    Rule 1: A=>B,C, conf = 0.43
    Rule 2: X=>Y, conf = 0.40

Each line in the <i>rules.csv</i> file must contain one rule alongside its support count, support, confidence, and lift. For example, A-B=>C,4,0.5,0.2,0.8 shows the support count, support, and confidence for the rule A-B=>C.

In [ ]:
# write the code to implement and mine the association rules here
# you can insert more cells below, if needed

<h2>Visualization (5 Points)</h2>

<p style="text-align: justify;">Draw a scatter plot based on the lift and confidence you saved in the rules.csv file.(x-axis=lift, y-axis=confidence) </p>

<p class="note">Note: Make sure the plot has title and axes names.</p>

In [8]:
# write the code to show a scatter plot of the association rules


<h2>Sensitivity Analysis (10 Points)</h2>


<p style="text-align: justify;">Run the frequent pattern mining function with the following values of minimum support and measure the total number of frequent patterns and the execution time. Next, draw three line plots using the matplotlib library that shows how minimum support (the x-axis) affects the runtime and the frequent itemsets (the y-axis). Note that x and y axes must have proper titles.
    
Minimum support values:
    
    0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01


In [9]:
# write the code here
# you can insert more cells below, if needed

<h3>Coding Considerations</h3>
<p style="text-align: justify;">You must consider the following in your implementations:</p>
<ul>
<li class="a"><span> The output CSV files should be generated in the same directory of your code when you run it. Use the naming provided in the Implementation section. </span>
<li class="a"><span> You should write your code in this Jupyter Notebook (*.ipynb) file which can generate the required reports itself.</span>
<li class="a"><span> Your code should be self-explanatory. Make sure you add comments and your output files are formatted correctly. You might lose up to 30 points for bad code quality (readability, modularity, comments, efficiency, etc.) and formatting of the output files.</span></li>
</ul>

<h2>Submission</h2>
<p style="text-align: justify;">Your python file or Jupyter Notebook file must generate all the abovementioned reports when you run it.</p>
<p style="text-align: justify;">You need to submit a zip file in Canvas, including the following items: 

<ul>
<li class="a"><span> a Jupyter Notebook (*.ipynb) file, named assignment3.ipynb that contains your completed code </span>
<li class="a"><span> patterns.csv that contains the frequent patterns</span>
<li class="a"><span> negative.csv that contains the negative corelated frequent itemsets.</span>
<li class="a"><span> maximal.csv that contains the maximal patterns</span>
<li class="a"><span> rules.csv that contains the association rules</span>
<li class="a"><span> results.pdf: a pdf file that contains all the requested charts and outputs (only copy the outputs (not the code itself) from your Notebook to this file. Also do not include the patterns and their supports in this file.)</span>
    
</li>
</ul>
    
<span style="background-color: #ffff99;">The file name should be in <strong>FirstName_LastName</strong> format</span>.</p>
<p style="text-align: justify;"><span style="background-color: #ffff99;">DO NOT INCLUDE EXTRA FILES, SUCH AS THE INPUT DATASETS</span>, in your submission;</p>
<p style="text-align: justify;">Please download your assignment after submission and make sure it is not corrupted or empty! We will not be responsible for corrupted submissions and will not take a resubmission after the deadline.</p>

<h2>Need Help?</h2>
<p>If you need help with this assignment, please get in touch with TAs (on MS Teams or via email) or go to their office hours.</p>

<p>You are highly encouraged to ask your question on the designated channel for Assignment 3 on Microsoft Teams (not necessarily monitored by the instructor/TAs). Feel free to help other students with general questions. However, DO NOT share your solution.</p>